In [1]:
import pandas as pd
import utils_HardnessMDL as utils_MDL

sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research


In [2]:
uci_ids = {
    "adult": 2,
    "contrac": 23,           # Contraceptive Method Choice
    "covertype": 31,
    "fertility": 244,
    "hill-valey": 606,
    "ht-sensor": 362,        # Gas sensors for home activity monitoring
    "iris": 53,
    "letter": 59,
    "libras": 181,
    "minibooone": 199,
    "skin": 229,
    "susy": 279,
    "wdbc": 17,              # Wisconsin Diagnostic Breast Cancer
    "wine": 109,
    "wine-red": 186,
    "wine-white": 187,
}

datasets = list(uci_ids.keys())

In [3]:
dataset_config = utils_MDL.load_dataset_config('dataset.yaml')



In [4]:
iris_config = dataset_config['iris']

iris_path = 'datasets/' + iris_config['file_name'][0]
iris_numerical_columns = iris_config['numeric_attributes']
iris_label_column = iris_config['class_attribute']

# TODO: Add proper process to categorical attributes
# TODO: Add process to multi file datasets

df_iris = pd.read_csv(iris_path)
measures_iris = utils_MDL.compute_kfold_description_lenght(df=df_iris, feature_cols=iris_numerical_columns, label_col=iris_label_column, k=10)
df_output_iris = pd.DataFrame(measures_iris).sort_values(by='index').reset_index(drop=True)
df_output_iris.to_csv(f"teste.csv", index=False)

sys.path: /home/emanu/ubuntu/ita/HardnessMDL/researchsys.path: /home/emanu/ubuntu/ita/HardnessMDL/researchsys.path: /home/emanu/ubuntu/ita/HardnessMDL/research

sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research

sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research
sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research
sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research
sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research
sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research
sys.path: /home/emanu/ubuntu/ita/HardnessMDL/research


In [5]:
iris_label_column

'class'

In [6]:
iris_config

{'file_name': ['iris.csv'],
 'n_instances': 150,
 'n_features': 4,
 'class_attribute': 'class',
 'numeric_attributes': ['sepal_length',
  'sepal_width',
  'petal_length',
  'petal_width'],
 'categorical_attributes': [],
 'target_classes': ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],
 'missing_values': False,
 'notes': 'No missing values'}